In [5]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
api_key = 

In [7]:
business_endpoint = 'https://api.yelp.com/v3/businesses/search'

In [8]:
header_category = {'Authorization': 'bearer %s' % api_key}

In [9]:
params_offset = 50

In [10]:
params_category_music = {'location': 'Nashville', 'limit': 50, 'categories': 'musicvenues', 'offset': params_offset}


In [11]:
response_business_music = requests.get(url = business_endpoint, params = params_category_music, headers = header_category)


In [12]:
response_business_music.json()

{'businesses': [{'id': '_XlYymFKwww0bGIgwvNbcA',
   'alias': 'the-bluebird-cafe-nashville',
   'name': 'The Bluebird Cafe',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/PMoiKtyjDMWEhlxj1QsBhg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/the-bluebird-cafe-nashville?adjust_creative=E6_YOL-L_jehSOPaHYyq0g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=E6_YOL-L_jehSOPaHYyq0g',
   'review_count': 376,
   'categories': [{'alias': 'musicvenues', 'title': 'Music Venues'},
    {'alias': 'tradamerican', 'title': 'American (Traditional)'}],
   'rating': 4.0,
   'coordinates': {'latitude': 36.1022, 'longitude': -86.8169999},
   'transactions': ['delivery', 'pickup'],
   'price': '$$',
   'location': {'address1': '4104 Hillsboro Pike',
    'address2': '',
    'address3': '',
    'city': 'Nashville',
    'zip_code': '37215',
    'country': 'US',
    'state': 'TN',
    'display_address': ['4104 Hillsboro Pike', 'Nashville, TN 37215']},
   'phon

In [13]:
def extract_music_info(music):
    categories =  [a['title'] for a in music['categories']]
    ratings = music['rating']
    name = music['name']
    address = music['location']['display_address']
    latitude = music['coordinates']['latitude']
    longitude = music['coordinates']['longitude']
    try:
        price = music['price']
    except:
        price = 'no price'
    zip_code = music['location']['zip_code']
    is_closed = music['is_closed']
    output_dict_music = { 
                   'categories': categories,   
                   'ratings': ratings,
                   'name': name,
                   'display_address': address,
                   'latitude': latitude, 
                   'longitude': longitude, 
                   'price': price, 
                   'zip_code': zip_code,
                   'is_closed': is_closed}

    return output_dict_music

In [14]:
music_list = []
for increment in range(0, 1000, 50) : 
    response_business_music = requests.get(url = business_endpoint, params = params_category_music, headers = header_category)
    for music_data in response_business_music.json()['businesses']:
        music_dict = extract_music_info(music_data)
        music_list.append(music_dict)
        params_offset += 50

KeyError: 'businesses'

In [171]:
music_venues_nash = pd.DataFrame(music_list)
music_venues_nash

""


In [150]:
music_venues_nash['display_address'] = music_venues_nash['display_address'].apply(lambda x: x[0] + x[1])
music_venues_nash


,categories,ratings,name,display_address,latitude,longitude,price,zip_code,is_closed
0,"[Music Venues, American (Traditional)]",4.0,The Bluebird Cafe,"4104 Hillsboro PikeNashville, TN 37215",36.102200,-86.817000,$$,37215,False
1,"[Bars, Music Venues]",4.0,The Local,"110 28th Ave NNashville, TN 37203",36.145910,-86.812620,$$,37203,False
2,"[Music Venues, American (Traditional), Wine Bars]",3.5,BB King's Blues Club,"152 2nd Ave NNashville, TN 37201",36.163349,-86.775960,$$,37201,False
3,"[Lounges, Music Venues]",4.5,Analog at Hutton Hotel,"1808 W End AveNashville, TN 37203",36.152860,-86.797080,$$,37203,False
4,"[American (New), Music Venues]",3.5,Commodore Grille,"2613 W End AveNashville, TN 37203",36.144580,-86.810830,$$,37203,False
...,...,...,...,...,...,...,...,...,...
795,[Music Venues],3.0,Vanderbilt Univ Langford Auditorium,"1 Garland AveNashville, TN 37240",36.143219,-86.804222,$,37240,False
796,[Music Venues],1.0,Nashville Center Stage,"514 8th Ave SNashville, TN 37203",36.153250,-86.778990,no price,37203,False
797,"[Music Venues, Juice Bars & Smoothies]",1.0,The Platform,"1500 2nd Ave SNashville, TN 37210",36.139003,-86.759543,no price,37210,False
798,[Music Venues],1.0,Time Travelers Cafe,"212 Louise AveNashville, TN 37203",36.152020,-86.804970,no price,37203,False


In [153]:
music_venues_nash['categories'] = music_venues_nash['categories'].apply(lambda x: x[0])
music_venues_nash

,categories,ratings,name,display_address,latitude,longitude,price,zip_code,is_closed
0,Music Venues,4.0,The Bluebird Cafe,"4104 Hillsboro PikeNashville, TN 37215",36.102200,-86.817000,$$,37215,False
1,Bars,4.0,The Local,"110 28th Ave NNashville, TN 37203",36.145910,-86.812620,$$,37203,False
2,Music Venues,3.5,BB King's Blues Club,"152 2nd Ave NNashville, TN 37201",36.163349,-86.775960,$$,37201,False
3,Lounges,4.5,Analog at Hutton Hotel,"1808 W End AveNashville, TN 37203",36.152860,-86.797080,$$,37203,False
4,American (New),3.5,Commodore Grille,"2613 W End AveNashville, TN 37203",36.144580,-86.810830,$$,37203,False
...,...,...,...,...,...,...,...,...,...
795,Music Venues,3.0,Vanderbilt Univ Langford Auditorium,"1 Garland AveNashville, TN 37240",36.143219,-86.804222,$,37240,False
796,Music Venues,1.0,Nashville Center Stage,"514 8th Ave SNashville, TN 37203",36.153250,-86.778990,no price,37203,False
797,Music Venues,1.0,The Platform,"1500 2nd Ave SNashville, TN 37210",36.139003,-86.759543,no price,37210,False
798,Music Venues,1.0,Time Travelers Cafe,"212 Louise AveNashville, TN 37203",36.152020,-86.804970,no price,37203,False


In [154]:
music_venues_nash.to_csv(r'C:\Users\mia\Desktop\nss\projects\capstone\data\nash_venues.csv', index = False)